# Eksploracja tekstów - Pracownia 2

## Zadanie 1

In [1]:
import pandas as pd
import numpy as np

file = open('tokenized_quotes.txt')
stopwords_file = open('polish.stopwords.txt')
formy_bazowe = pd.read_csv('polimorfologik-2.1.txt', sep = ';', header=None, usecols = [0,1])
formy_bazowe.columns = ['baza', 'inne']

In [2]:
import re

minicytaty = []
stopwords = []

for line in stopwords_file:
    clean_line = line.replace('\n', '')
    stopwords.append(clean_line)

for line in file:
    clean_line = line.replace('\n', '')
    clean_line = clean_line.replace('& lt ; /br & gt ;', '')
    clean_line = clean_line.replace('& lt ; br / & gt ;', '')
    clean_line = clean_line.replace('& lt ; br/ & gt ;', '')
    clean_line = clean_line.replace('& lt ; br & gt ;', '')
    clean_line = clean_line.replace('& lt ;', '')
    clean_line = clean_line.replace('& gt ;', '')
    clean_line = clean_line.replace('& quot ;', '')
    clean_line = clean_line.replace("''", '')
    minicytaty.append(clean_line.split('.'))

##### Dodatkowe wyrazy, które uważam, że mogą "psuć" zapytanie:

In [3]:
stopwords.append('to')
stopwords.append('czym')
stopwords.append('siebie')
stopwords.append('kim')

In [4]:
minicytaty = [val for sublist in minicytaty for val in sublist] # łączenie w jedną listę
minicytaty = list(filter(None, minicytaty)) # usuwanie pustych elementów
minicytaty = list(filter(lambda x: x.islower(), minicytaty)) # usuwanie wyrazów, które składają się tylko z cyfr lub znaków interpunkcyjnych
minicytaty = [x.split() for x in minicytaty] # lista list, tokenizacja
minicytaty = list(np.unique(minicytaty)) # pozbywamy się takich samych cytatów

##### Żeby uniknąć cytatów, których znaczenie zależy od kontekstu, znów filtrujemy naszą listę:

In [5]:
words = ['on', 'ona', 'go', 'ich', 'jej', 'jego']
minicytaty = [cytat for cytat in minicytaty if not any(x in cytat for x in words)]  # zostawiamy tylko te, które nie mają w sobie 'words'

##### Cytaty, które zawierają znaki interpunkcyjne (inne niż przecinek lub kropka) są najczęściej dialogami i nie wyglądają dobrze

In [6]:
punctuation = ["'", "(", ")", "-", "!", "[", "]", "–", "", "&", "?", "''", ":"]
minicytaty = [cytat for cytat in minicytaty if not any(p in cytat for p in punctuation)]

In [7]:
minicytaty = [cytat for cytat in minicytaty if len(cytat) <= 30]

##### Funkcja pomocnicza, usuwająca stopwords z zapytania (przesiewamy zapytanie, żeby zostawić (najlepiej) głównie rzeczowniki lub/i czasowniki):

In [8]:
def stopwords_del(seq, stopwords):
    temp_seq = re.split('\W', seq)
    temp_seq = list(filter(None, temp_seq))
    idx = []
    for i in range(len(temp_seq)):
        if temp_seq[i] in stopwords:
            idx.append(i)
    for i in sorted(idx, reverse = True):
        del temp_seq[i]
    return(temp_seq)

In [9]:
def MCR(baza = minicytaty, stopwords = stopwords, losowanie = True, lematyzacja = False):
    print('Jeśli chcesz zakończyć rozmowę z Mistrzem Ciętej Riposty, wpisz "q".')
    idx = [] # indeksy dotychczasowych odpowiedzi
    while True:
        quest = input('JA: ').lower()
        if quest == 'q':
            break
        
        freq = {} # klucz: indeks, wartość: ile razy wybrano cytat o tym indeksie
        probs = {}
        
        # zostawiamy tylko 'kluczowe' słowa
        quest_lst = stopwords_del(quest, stopwords)
        #print(quest_lst)
        
        # to się przyda do porównywania czy odp. nie jest 'kopią' zapytania
        quest = re.split('\W', quest)
        quest = list(filter(None, quest))
        
        # z lematyzacją
        if lematyzacja:
            for q in quest_lst:
                # q -> forma bazowa q -> wszystkie odmiany formy bazowej q
                q_odmiany = list(formy_bazowe.loc[formy_bazowe['baza'] == 
                                                  ''.join(list(formy_bazowe.loc[formy_bazowe['inne']  == q]['baza']))]['inne'])
                # jeśli nie ma odmian tego wyrazu:
                if len(q_odmiany) == 0:
                    q_odmiany = [q]
                    
                for word in q_odmiany:
                    i = 0
                    for cytat in baza:
                        if word in cytat:
                            if i in freq:
                                freq[i] += 1
                            else:
                                freq[i] = 1
                        i += 1
        else:
            for q in quest_lst:
                i = 0
                for cytat in baza:
                    if q in cytat:
                        if i in freq:
                            freq[i] += 1
                        else:
                            freq[i] = 1
                    i += 1
        
        sort_freq = {k: v for k, v in sorted(freq.items(), key=lambda item: item[1], reverse=True)}
        if len(freq) == 0:
            print('MCR: Hmm nie wiem jak Ci na to odpowiedzieć')
        
        elif losowanie:
            s = sum(sort_freq.values())
            probs = {k: v/s for k, v in sort_freq.items()}
            i = 0
            while True:
                answer_idx = int(np.random.choice(list(probs), 1, list(probs.values())))
                
                if (answer_idx not in idx) and (not all(x in baza[answer_idx] for x in quest)):
                    print('MCR: ',' '.join(baza[answer_idx]))
                    break
                    
                # jeśli dana odp juz była lub jest w pewnym sensie 'kopią' poprzednika, losujemy jeszcze raz
                elif i < 10:
                    i += 1
                    
                else:
                    print('MCR: ',' '.join(baza[answer_idx]))
                    break
        
        # bez losowania -> lider wygrywa
        else:
            j = 0
            while True:
                answer_idx = list(sort_freq)[j]
                
                # pilnujemy zeby wypowiedzi w rozmowie się nie powtarzały + żeby odpowiedź nie była podobna do naszego zdania
                if (answer_idx not in idx) and (not all(x in baza[answer_idx] for x in quest)):
                    print('MCR: ',' '.join(baza[answer_idx]))
                    break
                
                elif j < len(freq) - 1:
                    j += 1
                    
                # ewentualna możliwość na powtórzenie cytatu: wszystkie inne w tym kontekście były już wywołane
                else:
                    print('MCR: ',' '.join(baza[next(iter(sort_freq))]))
                    break
            idx.append(answer_idx)

In [47]:
MCR(losowanie = True, lematyzacja = True)

Jeśli chcesz zakończyć rozmowę z Mistrzem Ciętej Riposty, wpisz "q".


JA:  jak dbać o kondycję?


MCR:  coraz więcej aktorów potrafi w zachodni sposób dbać o swoją karierę


JA:  a kondycja?


MCR:  ci , którzy to robią , nie mówią o tym , a ci , którzy o tym mówią , nie robią tego


JA:  cwaniaczki


MCR: Hmm nie wiem jak Ci na to odpowiedzieć


JA:  q


In [48]:
MCR(losowanie = False, lematyzacja = False)

Jeśli chcesz zakończyć rozmowę z Mistrzem Ciętej Riposty, wpisz "q".


JA:  jak dbać o kondycję?


MCR:  aby dalej ugruntować zdrowie naszego narodu , musimy dbać o wychowanie naszej młodzieży


JA:  q


In [11]:
# def MCR(baza, losowanie = False):
#     print('Jeśli chcesz zakończyć rozmowę z Mistrzem Ciętej Riposty, wpisz "q".')
#     freq = {} # klucz: indeks, wartość: ile razy wybrano cytat o tym indeksie
#     while True:
#         quest = input('JA: ').lower()
#         if quest == 'q':
#             break
        
#         freq = {}
#         quest_lst = stopwords_del(quest, stopwords)
#         quest_lst = list(filter(None, quest_lst))
#         #cytaty_dict = {}
# #         for cytat in baza:
# #             if all(x in cytat for x in quest_lst): # całe wyrażenie
# #                 if ' '.join(quest_lst) in cytaty_dict: # jeśli jest już taki klucz w słowniku
# #                     cytaty_dict[' '.join(quest_lst)].append(' '.join(cytat))
# #                 else:
# #                     cytaty_dict[' '.join(quest_lst)] = [' '.join(cytat)]  
                    
#         for q in quest_lst:
#             i = 0
#             for cytat in baza:
#                 if q in cytat:
# #                     if q in cytaty_dict: # jeśli jest już taki klucz w słowniku
# #                         cytaty_dict[q].append(' '.join(cytat))
# #                         freq[i] += 1
# #                     else:
# #                         cytaty_dict[q] = [' '.join(cytat)]
# #                         freq[i] = 1
#                     if i in freq:
#                         freq[i] += 1
#                     else:
#                         freq[i] = 1
#                 i += 1
                        
#         #cytaty_df = pd.DataFrame([(key, var) for (key, L) in cytaty_dict.items() for var in L], columns = ['term', 'cytat'])
#         #cytaty_df.groupby('cytat')
        
#         # na górze będą wyrazy występujące najczęściej
#         #duplicate = cytaty_df[cytaty_df.duplicated(['cytat'])]
#         #duplicate[['cytat']]
#         sort_freq = {k: v for k, v in sorted(freq.items(), key=lambda item: item[1], reverse=True)}
        
#         if len(freq) == 0:
#             print('MCR: Hmm nie wiem jak Ci na to odpowiedzieć')
#         else:
#             #duplicateRowsDF[duplicateRowsDF["name"] == "World"].tolist()[0]
#             #print('MCR: ', duplicate.iloc[0]['cytat'])
#             print('MCR: ',' '.join(minicytaty[next(iter(sort_freq))])) # pierwszy klucz ze słownika posortowanego wg wartości
        
# #         if cytaty_df.shape[0] == 0:
# #             print('Hmm nie wiem jak Ci na to odpowiedzieć')
# #         else:
# #             print('MCR: ', list(cytaty_dict.values())[0][0])

In [12]:
#człon = ''.join(list(formy_bazowe.loc[formy_bazowe['inne']  == 'absbd']['baza']))
#list(formy_bazowe.loc[formy_bazowe['baza'] == ''.join(list(formy_bazowe.loc[formy_bazowe['inne']  == 'babciu']['baza']))]['inne'])
#człon

In [13]:
#dict = {1: 2, 3:4}
#sum((dict.values()))